In [1]:
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import urllib.request, json
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
from tqdm import tqdm

Generic class to store an image and its IIIF representation

In [2]:
class Terzani_Photo(object):
    def __init__(self, iiif, country):
        self.iiif = iiif
        self.country = country
        
    def get_photo_link(self):
        return self.iiif["images"][0]["resource"]["@id"]

First we need to know all the collections for the Terzani archive. For this we scrape the HTML content from the Fondo Tiziano Terzani (collection 1352). We store the collections number in the ```collections``` list. We just need to discard collections 1353, 1360 and 1450 which are collections of collections

In [3]:
r = requests.get('http://dl.cini.it/collections/show/1353') # /ip: Returns the requester's IP Address.
soup = BeautifulSoup(r.text, "html.parser")
collections = []
unsupported_collections = ['1352', '1360', '1450']
for a in soup.find_all("a", href=True):
    link = a['href']
    if link.startswith("/collections/show/") and link[-4:] not in unsupported_collections:
        collections.append(link[-4:])
    
#Preview
collections[:5]
collections_country = dict.fromkeys(collections, None)
collections_country[1460] = "China"
collections_country[1465] = "China"
for i in range(1451, 1460):
    collections_country[i] = "China"
collections_country[1444] = "Japan"
collections_country[1404] = "South Africa"
collections_country[1446] = "Cambodia"
collections_country[1413] = "India"
collections_country[1414] = "India"
collections_country[1415] = "India"
collections_country[1416] = "India"
collections_country[1420] = "India"
collections_country[1418] = "South Korea"
collections_country[1419] = "Laos"
collections_country[1421] = "Philippines"
collections_country[1423] = "Tibet"
collections_country[1425] = "India"
collections_country[1424] = "Nepal"
collections_country[1426] = "Nepal"
collections_country[1427] = "Singapore"
collections_country[1429] = "Japan"
collections_country[1428] = "Japan"
collections_country[1433] = "Vietnam"
collections_country[1431] = "Vietnam"
collections_country[1432] = "Vietnam"
collections_country[1434] = "Myanmar"
collections_country[1435] = "Myanmar"
collections_country[1437] = "Japan"
collections_country[1403] = "Vietnam"
collections_country[1440] = "Cambodia"
collections_country[1442] = "Cambodia"
collections_country[1439] = "Cambodia"
collections_country[1441] = "Cambodia"
collections_country[1449] = "Cambodia"
collections_country[1438] = "Cambodia"

Now we can read the manifest of every collection. For every entry of the manifest, we only keep it if it is the recto of a photo

We check if an picture is color or black and white and store them seperately.

In [4]:
terzani_recto_iiif_color = list() #to store color photos in Terzani photo collection
terzani_recto_iiif_bw = list() #to store black and white photos in Terzani photo collection

threshold_color = 0.2

iiif_format = "/full/max/0/default.jpg"

for collection in tqdm(collections):
    url = "http://dl.cini.it/oa/collections/"+collection+"/manifest.json"

    response = urllib.request.urlopen(url)

    data = json.loads(response.read())

    for entry in tqdm(data["sequences"][0]["canvases"]):
        if entry["label"] == None:
            continue
        if  entry["label"].lower().endswith("recto"):
            photo_link = entry["images"][0]["resource"]["service"]["@id"] + iiif_format
            photo = np.asarray(bytearray(urllib.request.urlopen(photo_link).read()), dtype="uint8")
            photo = cv2.imdecode(photo, cv2.IMREAD_COLOR)
            b,g,r = cv2.split(photo)
            isize = r.size
            rg = np.abs(r-g)
            rb = np.abs(r-b)
            gb = np.abs(g-b)
            diff =np.sum(rg+rb+gb)
            diff = diff / isize / (255 * 3)
            if diff > threshold_color: # As gray values have r-g = 0 and r-b = 0 and g-b = 0 diff will be near 0 for grayscale images and > 0 for colored images.
                terzani_recto_iiif_color.append(Terzani_Photo(entry, collections_country[collection]))
            else:
                terzani_recto_iiif_bw.append(Terzani_Photo(entry, collections_country[collection]))

 25%|████████████████████▏                                                           | 130/515 [01:38<05:03,  1.27it/s]


 51%|█████████████████████████████████████████                                       | 264/515 [03:18<03:21,  1.25it/s]


 77%|█████████████████████████████████████████████████████████████▌                  | 396/515 [04:56<01:27,  1.35it/s]


  5%|███▊                                                                              | 9/192 [00:02<01:06,  2.77it/s]


 73%|██████████████████████████████████████████████████████████▊                     | 141/192 [00:44<00:18,  2.83it/s]


...

 73%|███████████████████████████████████████████████████████████▊                      | 35/48 [00:31<00:12,  1.07it/s]


 68%|██████████████████████████████████████████████████████                          | 115/170 [01:41<00:49,  1.11it/s]


 26%|████████████████████▋                                                            | 47/184 [00:42<02:02,  1.12it/s]


 97%|█████████████████████████████████████████████████████████████████████████████▊  | 179/184 [02:38<00:04,  1.15it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 59/59 [3:42:49<00:00, 226.59s/it]


In [5]:
len(terzani_recto_iiif_color)

1354

In [6]:
len(terzani_recto_iiif_bw)

7170

Let's now dump this list in a pickle file

In [7]:
with open('terzani_recto_iiif_color.pickle', 'wb') as file:
    pickle.dump(terzani_recto_iiif_color, file)

In [8]:
with open('terzani_recto_iiif_bw.pickle', 'wb') as file:
    pickle.dump(terzani_recto_iiif_bw, file)